# TALENT Course 11
## Learning from Data: Bayesian Methods and Machine Learning
### York, UK, June 10-28, 2019 

## Exercise: Bayesian neural networks
Adapted from the [blog entry](http://krasserm.github.io/2019/03/14/bayesian-neural-networks/) by Martin Krasser with some modifications. See also the lecture notes that accompany this execise and the following reference:
* [Weight Uncertainty in Neural Networks](https://arxiv.org/abs/1505.05424) (*Bayes by Backprop*). 

Additional references for Bayesian Neural Networks:
* [Tensorflow Probability](https://www.tensorflow.org/probability)
* [Keras](https://keras.io/)
* [Edward](http://edwardlib.org/)

You will need tensorflow and keras for running this lecture notebook and the accompanying exercise. In this directory, please run

    conda deactivate
    conda env create
   
which reads the environment.yml file and creates the virtual environment talent-env-ml. It can be activated:

    conda activate talent-env-ml

## Implementation example

Variational inference of neural network parameters is now demonstrated on a simple regression problem. We therefore use a Gaussian distribution for $p(y \lvert \mathbf{x},\mathbf{w})$. The training dataset consists of a few noisy samples `x_train`, `y_train` drawn from the `true_function` function with Gaussian noise.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

import seaborn as sns; 
sns.set('talk')

# make your loops show a smart progress meter - just wrap any iterable with tqdm(iterable)
import tqdm

#### Subtask
* Insert the code cell from the lecture that defines the true function and constructs the training data.
* Insert the code cell that creates the training data.
* Insert the code cell that produces an array with true function evaluations.

In [5]:
# Insert code here
#
#

#### Questions
* Plot the true function and the training data together in a figure.
* How does the true function behave outside of the region of predictor variables (`x_train`)?
* How do you think that the prediction of the BNN will look like in the interpolation and extrapolation regions, respectively?

In [5]:
# Insert code here
#
#

The noise in training data gives rise to aleatoric uncertainty. To cover epistemic uncertainty we implement the variational inference logic in a custom `DenseVariational` Keras layer. The learnable parameters of the mixture prior, $\sigma_1$ $\sigma_2$ and $\pi$, are shared across layers. The complexity cost (`kl_loss`) is computed layer-wise and added to the total loss with the `add_loss` method. Implementations of `build` and `call` directly follow the equations defined above. 

Note that many of these features are now available in the [Tensorflow Probability](https://www.tensorflow.org/probability) Python library. It might be instructive, however, to see how the DenseVariational Layer is constructed explicitly.

#### Subtasks
* Insert the code cell that defines the mixture priors and the `DenseVariational` Keras layer.
* Insert the code cell that defines the Keras Input and Model.
* Note that you will probably get several deprecation warnings. They disappear if you run the cell a second time...

In [5]:
# Insert code here
#
#

Our model is a neural network with two `DenseVariational` hidden layers, each having 20 units, and one `DenseVariational` output layer with one unit. Instead of modeling a full probability distribution $p(y \lvert \mathbf{x},\mathbf{w})$ as output the network simply outputs the mean of the corresponding Gaussian distribution. In other words, we do not model aleatoric uncertainty here and assume it is known. We only model epistemic uncertainty via the `DenseVariational` layers.

Since the training dataset has very few data points we train the network with all of them per epoch so that the number of batches per epoch is 1. For other configurations, the complexity cost (`kl_loss`) must be weighted by $1/M$ as described in section 3.4 of the [paper](https://arxiv.org/abs/1505.05424) where $M$ is the number of mini-batches per epoch.

#### Questions
* Print a summary of the model using the command `model.summary()`.
* Make a simple drawing of the neural network indicating the different layers and their respective units. 

  *Hint: the input and output layers have one unit while the hidden layers have several.*.
* Can you understand the number of parameters at each layer? 

  *Hint 1: For neurons, every incoming signal is multiplied by a weight while the output is also controlled by a bias.*

  *Hint 2: Remember that each weight and bias is parametrized by a mean and a standard deviation. In addition there are three common parameters per layer [$\sigma_1$ $\sigma_2$ and $\pi$].*
  
* How many parameters would this network have if it was not a probabilistic one (i.e. point estimates for all weights and biases)?

In [5]:
# Insert code here
#
#

The network can now be trained with a Gaussian negative log likelihood function (`neg_log_likelihood`) as loss function assuming a fixed standard deviation (`noise`). This corresponds to the *likelihood cost*, the last term in equation $3$. 

#### Subtasks
* Insert the code cell that defines the negative log-likelihood. (Again you might get some deprecation warning.)
* Insert the code cell that performs the model fit. Modify it so that it only runs for 500 epochs.

In [5]:
# Insert code here
#
#

#### Subtasks / questions
* Insert the code cell that makes 500 predictions at positions given by an array `X_test`. 
* What does it mean to make a sample (a prediction) with this Bayesian neural network? Explain in terms of the network parameters (weights and biases).
* Insert the code cell that extracts the model prediction in terms of a mean and a standard deviation.
* Insert the code cell that produces a plot of the model prediction, the training data, and the true function.
* Have the training epochs been sufficient for th network to capture the structure of the training data?
* Does the width of the uncertainty bands in different regions make sense? Why?

In [5]:
# Insert code here
#
#

#### Subtasks / questions
* Repeat the training for 10000 epochs and plot again the model prediction, the training data, and the true function.
* Are these training epochs been sufficient for the network to capture the structure of the training data?
* How does the mode look like in the extrapolation region?
* If you have time: extend the training region, change the number of training data and/or the error.
* If you have time: Can you figure out how to extract e.g. a single weight or bias for different instances of the network? It would be interesting to plot this distribution as a function of the number of training epochs. Admittedly, I (CF) did not yet figure out myself how to perform this task.

In [5]:
# Insert code here
#
#

For an example how to model both epistemic and aleatoric uncertainty I recommend reading [Regression with Probabilistic Layers in TensorFlow Probability](https://medium.com/tensorflow/regression-with-probabilistic-layers-in-tensorflow-probability-e46ff5d37baf) which uses probabilistic Keras layers from the upcoming Tensorflow Probability 0.7.0 release. Their approach to variational inference is similar to the approach described here but differs in some details. For example, they compute the complexity cost analytically instead of estimating it from Monte Carlo samples, among other differences.